1. Pobierz dane wejściowe.

In [37]:
import time
import pandas as pd
import requests
import json
from datetime import datetime, timedelta

# Oblicz pierwszy i ostatni dzień zeszłego miesiąca
today = datetime.today()
first_day_of_current_month = datetime(today.year, today.month, 1)
last_day_of_last_month = first_day_of_current_month - timedelta(days=1)
first_day_of_last_month = datetime(last_day_of_last_month.year, last_day_of_last_month.month, 1)

# Konwertuj na UNIX timestamp
first_day = int(first_day_of_last_month.timestamp())
last_day = int(first_day_of_current_month.timestamp())

# Pobierz dane z API
headers = {
    'X-BLToken': 'TOKEN'
}
container = []

while True:
    data = {
        'method': 'getInvoices',
        'parameters': f'{{"date_from": {first_day}}}'
    }
    response = requests.post('https://api.baselinker.com/connector.php', headers=headers, data=data)
    baza = response.json().get('invoices', [])
    if not baza:
        break
    container.extend(baza)
    first_day = baza[-1]['date_add']
    if first_day > last_day:
        break
    time.sleep(1)

df = pd.DataFrame(container)

# Usuń niepotrzebne kolumny
df = df[df['date_add'] < last_day]

# Usuń duplikaty
df.drop_duplicates(subset=['invoice_id'], inplace=True)

2. Sprawdź, czy dane są poprawne.

In [41]:
print('Zakres od dnia: ' + str(first_day_of_last_month))
print('Zakres do dnia: ' + str(first_day_of_current_month))
print('Liczba faktur: ' + str(len(df)))

df.info()

Zakres od dnia: 2024-08-01 00:00:00
Zakres do dnia: 2024-09-01 00:00:00
Liczba faktur: 1082
<class 'pandas.core.frame.DataFrame'>
Index: 1082 entries, 0 to 1091
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   invoice_id                1082 non-null   int64  
 1   order_id                  1082 non-null   int64  
 2   series_id                 1082 non-null   int64  
 3   type                      1082 non-null   object 
 4   number                    1082 non-null   object 
 5   year                      1082 non-null   int64  
 6   month                     1082 non-null   int64  
 7   sub_id                    1082 non-null   int64  
 8   postfix                   1082 non-null   object 
 9   date_add                  1082 non-null   int64  
 10  date_sell                 1082 non-null   int64  
 11  date_pay_to               1082 non-null   int64  
 12  currency                  1082 

3. Zapisz dane do pliku XLSX

In [29]:
df.to_excel('invoices.xlsx', index=False)